Se analizará una base de datos de Kaggle con información de población de varios paises del mundo.

In [ ]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt

Haremos la conexión con la base de datos

In [ ]:
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        port=3306,
        user="root",
        password="1234",
        database="POBLACION"
    )

def query_db(query):
    connection = connect_to_db()
    cursor = connection.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    columns = [col[0] for col in cursor.description]
    df = pd.DataFrame(data, columns=columns)
    cursor.close()
    connection.close()
    return df


Crearemos una tabla con los datos de México

In [ ]:
query_1 = '''
    SELECT * FROM Poblacion_global
    WHERE Country_Name = "Mexico"
'''
df_mx = query_db(query_1)

df_mx

Podemos ver cuanta gente ha tenido 80 años o más a lo largo de los años

In [ ]:
df_mx_80 = df_mx[['Indicator_Name']==['Population ages 80 and above, male','Population ages 80 and above, female']]
df_mx_80

Y podemos hacer una tabla comparativa de hombres y mujeres.

In [ ]:
df_mx_80.drop(columns=['Country_Name','Country_Code','Indicator_Name'])
x = [i for i in range(1960,2023,1)]
f_80 = df_mx_80.iloc[0]
m_80 = df_mx_80.iloc[1]

plt.plot(x,f_80,'bo',m_80,'ro')


Una pregunta interesante podria ser, ¿cuántos adultos mayores a 60 años hay en el año 2019 y cuántos en el año 2020? Esto para ver que tanto afecto la pandemia.

In [ ]:
query_2 = '''
SELECT  Indicator_Name, SUM(2019), SUM(2020) FROM Poblacion_global
WHERE Country_Name = "Mexico" AND Indicator_Name = "Population ages 80 and above, male"
OR Country_Name = "Mexico" AND Indicator_Name = "Population ages 80 and above, female"
OR Country_Name = "Mexico" AND Indicator_Name = "Population ages 70 and above, male"
OR Country_Name = "Mexico" AND Indicator_Name = "Population ages 70 and above, female"
OR Country_Name = "Mexico" AND Indicator_Name = "Population ages 60 and above, male"
OR Country_Name = "Mexico" AND Indicator_Name = "Population ages 80 and above, female"
'''

df_mx_mayores  = query_db(query_2)

df_mx_mayores.iloc[0,1] 